In [1]:
%display latex
from pylie import *
from collections.abc import Iterable
import functools
from operator import mul, sub
from pprint import pprint

In [2]:
def reduce(e1: Differential_Polynomial,e2: Differential_Polynomial, context:Context)->Differential_Polynomial:
    # XXX except only function names in context list
    assert e2.is_monic()
    def _order (der):
        if der != 1:
            return pylie.order_of_derivative(der)
        else :
            return [0]*len(context._independent)
    print ("--------------------------------------- entering reduce")
    e1._orig.show()
    e2._orig.show()
    ld = e2.Lder()
    e2_order = _order (ld)    
    for t, d, c, in zip (e1.terms(), e1.derivatives(), e1.coefficients()):
        e1_order = _order (d)
        dif = []
        for a, b in zip (e1_order, e2_order):
            dif += [a - b]
        try:
            if ld.operator().function() != d.operator().function():
                continue
        except AttributeError:
            continue            
        if all (map (lambda h: h == 0, dif)) :
            # XXX: do we need that in homogenous environmeent?
            e3 = sum(_ for _ in e1.terms()) - c * sum (_ for _ in e2.terms())
            dp = Differential_Polynomial(e3, context)
            assert dp.is_monic()
            return dp
        elif  all (map (lambda h: h >= 0, dif)):
            print ("PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP")
            print ("PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP")
            print ("PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP")
            # get variables which shuld be differentiate
            variables_to_diff = []
            for i in range (len(context._independent)):
                if dif [i] != 0:
                    variables_to_diff.extend ([context._independent[i]]*abs(dif[i]))
            print ("Vorher", variables_to_diff)
            print ("reduce {} with {}".format(e1.Lder(), e2.Lder()))
            print ("e1:", e1.Lterm())
            print ("e2:", e2.Lterm())
            e1._orig.show()
            e2._orig.show()
            print ("after derivation................")
            diff(e2._orig, *variables_to_diff).show()
            e3 = e1._orig - c * diff(e2._orig, *variables_to_diff)
            print ("computation done 1")
            e3.show()
            dp =  Differential_Polynomial (e3, context)
            #print ("e3:", dp.Lterm())            
            #dp._orig.show()
            #    print (":3", _._d, _._coeff)            
            return dp
        else:
            print ("no reduction?")
            pass
    return e1

In [3]:
#indeterminates = (x, y) = var("x y")
#R=PolynomialRing(QQ, indeterminates)
#z  = function ("z")(*indeterminates)
#ctx = Context((z,), indeterminates)
#print("*"*70)
#e1 = diff(z,y)-x**2/y**2*diff(z,x)-(x-y)*z(x,y)/y**2
#e1_dp = Differential_Polynomial (e1, ctx)
#print (e1_dp)
#print("*"*70)
#f1 = diff(z, x) + z(x,y)/x
#f1_dp = Differential_Polynomial (f1, ctx)
#f1_dp.is_monic()
#print (f1_dp)
print("*"*70)
#f2 = diff(z, y) + z(x,y)/y
#f2_dp = Differential_Polynomial (f2, ctx)
#print (f2_dp)
#r1 = reduce (e1_dp, f1_dp, ctx)
#r1
#r1._orig
#r2 = reduce (r1, f2_dp, ctx)

**********************************************************************


In [4]:
def Lfunc(e):
    return e

In [5]:
def Lcml(e):
    '''Least common left multiply, p.23'''
    return 0

In [6]:
def Lcm(e):
    return 0

In [7]:
def CompleteSystem(S, context):
    return S

In [8]:
def FindIntegrableConditions(S, context):
    return S

In [9]:
def ReduceIntegrabilityConditions(S, context):
    return S

In [10]:
def Reorder (S, context, ascending = False):
    res=sorted(S)
    if ascending :
        res.reverse()
    return res

In [11]:
var ("x y")
z = function("z")(x,y)
w = function("w")(x,y)
ctx = Context((w,z), (x,y), MatrixOrder.Mgrlex)
f1 = Differential_Polynomial(diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w(x,y)/y, ctx)
f1._orig

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

In [12]:
f2=Differential_Polynomial(diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x,ctx)
f2._orig

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

In [13]:
f3 = Differential_Polynomial(diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2 ,ctx)
f3._orig

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

In [14]:
f4 = Differential_Polynomial(diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w(x,y)/(2*y**2), ctx)
f4._orig
f4._p

[<pylie.JanetBasis.DTerm object at 0x7f1a6fcc7220>,
 <pylie.JanetBasis.DTerm object at 0x7f1a69e211f0>,
 <pylie.JanetBasis.DTerm object at 0x7f1a6fcc7580>,
 <pylie.JanetBasis.DTerm object at 0x7f1a6a89d400>,
 <pylie.JanetBasis.DTerm object at 0x7f1a69cfeac0>,
 <pylie.JanetBasis.DTerm object at 0x7f1a6a89d250>]

In [15]:
f5 = Differential_Polynomial(diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w(x,y)/y**2, ctx)
f5._orig

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

In [16]:
def Autoreduce(S, context):  
    i = 0
    while True:
        p = S[i]
        print ("While")
        p._orig.show()  
        r = S[i+1:]
        if not r:
            S = Reorder(S, context,True)
            return S[:]    
        r[0]._orig.show()
        have_reduced = False
        checkidx = i+1
        for _r in r:
            print ("Try to reduce r{} with r{}".format(checkidx, i))
            res = reduce (_r, p, context)
            if res._orig != _r._orig:
                print ("have reduced!")
                res._orig.show()
                for gurki in res._p:
                    print (gurki)
                have_reduced = True                
            S[checkidx] = res
            checkidx += 1
        S = Reorder(S, context, ascending = True)
        print("This is the new System........")
        for _ in S:
            _._orig.show()
        if not have_reduced:
            # no reduction done
            i += 1        
        else:
            i = 0
            

In [17]:
def JanetBasis(S, context):
    """List of homogenous PDE's + order context"""
    if not isinstance(S, Iterable):
        # bad criterion
        S = [S]
    S = [Differential_Polynomial(s, context) for s in S]
    S = Reorder(S, context, ascending = True)
    for _ in S:
        _._orig.show()
    while 1:
        S = Autoreduce (S, context)
        #S = CompleteSystem (S, context)
        #f = FindIntegrableConditions(S, context)
        #c = ReduceIntegrabilityConditions(S, context)
        #if all ([_ == 0 for _ in c]) or not c:
        #    return S
        #S += c
        #S = Reorder (S, context)
        return S[:]

In [ ]:
checkS=JanetBasis([f3._orig,f1._orig,f5._orig, f2._orig,f4._orig], ctx)

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

...

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

-1/2*y*(x*diff(z(x, y), y)/(x^2*y + y^2) - 2*w(x, y)/y + 2*diff(w(x, y), y))/x + y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + 17/2*(2*x^2*w(x, y) - 2*y^2*diff(w(x, y), y) - 2*(x^2*diff(w(x, y), y) - w(x, y))*y - x*diff(z(x, y), y))/(x^3 + x*y) + diff(z(x, y), x, y)

Try to reduce r1 with r0
--------------------------------------- entering reduce


-1/2*y*(x*diff(z(x, y), y)/(x^2*y + y^2) - 2*w(x, y)/y + 2*diff(w(x, y), y))/x + y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + 17/2*(2*x^2*w(x, y) - 2*y^2*diff(w(x, y), y) - 2*(x^2*diff(w(x, y), y) - w(x, y))*y - x*diff(z(x, y), y))/(x^3 + x*y) + diff(z(x, y), x, y)

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

In [ ]:
for i in (f1._orig, f2._orig,f3._orig, f4._orig,f5._orig):
    i.show()

In [ ]:
for _ in checkS:
    _._orig.show()

In [ ]:
g1 = Differential_Polynomial (diff(z,y,y) + diff(z,y)/(2*y), ctx)
g1._orig

In [ ]:
g2 = Differential_Polynomial(diff(w,x,x) + 4*y*y*diff(w,y) - 8*y*y*diff(z, x) - 8*y*w(x,y), ctx)
g2._orig

In [ ]:
g3 = Differential_Polynomial(diff(w,x,y) - diff(z,x,x)/2 - diff(w,x)/(2*y) - 6*y**2*diff(z,y), ctx)
g3._orig

In [ ]:
g4 = Differential_Polynomial(diff(w,y,y) - 2*diff(z,x,y)- diff(w,y)/(2*y) + w(x,y)/(2*y**2), ctx)
g4._orig

In [ ]:
f1._orig

In [ ]:
f4._orig

In [ ]:
print(f4.Lder(), f1.Lder())

In [ ]:
f4a = reduce(f4, f1, ctx)

In [ ]:
for _ in f4a._p:
   print (_)

In [ ]:
f4a._orig

In [ ]:
wuwu=f4a._p[-1]._coeff.operands()[0]*f4a._p[-1]._coeff.operands()[1].expand().simplify_full()

In [ ]:
f4b = reduce(f4a, f1, ctx)

In [ ]:
f4b._orig

In [ ]:
functools.reduce(operator.mul, f4b._p[1]._coeff.operands()).expand().simplify_full()

In [ ]:
f4c = reduce(f4b, f2, ctx)

In [ ]:
f4c._p[2]._coeff.expand().simplify_full()

In [ ]:
f4d = reduce (f4c, f1, ctx)

In [ ]:
f4d._p[1]._coeff.expand().simplify_full()

In [ ]:
f4d._p[0]._d

In [ ]:
wuwu.collect(y)
# correct according Example 2.35

In [ ]:
wuwu=f4a._p[-2]._coeff

In [ ]:
wuwu

In [ ]:
wuwu=f4a._p[-2]._coeff.operands()

In [ ]:
wuwu

In [ ]:
wawa=wuwu[0]*wuwu[1]

In [ ]:
wawa.expand().simplify_full()

In [ ]:
# wawa is a